# Competição Kaggle: Natural Language Processing with Disaster Tweets


## Abordagem 1: TF-IDF + Regressão Logística

Este notebook implementa a primeira abordagem utilizada na competição, que consiste em:

1. **Pré-processamento de Texto:** Limpeza básica dos tweets.
2. **Vetorização TF-IDF:** Conversão do texto em uma matriz numérica usando Term Frequency-Inverse Document Frequency.
3. **Modelo de Regressão Logística:** Treinamento de um classificador simples e eficiente.

Esta abordagem obteve o melhor score (0.79313) entre todas as tentativas.

### 1. Instalação das Bibliotecas Necessárias

Execute esta célula para instalar as bibliotecas necessárias. Se você já as tem instaladas, pode pular esta etapa.

In [1]:
!pip install pandas scikit-learn nltk numpy

### 2. Importação das Bibliotecas e Configuração Inicial

Importamos todas as bibliotecas necessárias e configuramos o ambiente.

In [2]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
import pickle
import os

# Baixar recursos do NLTK (stopwords) se ainda não tiver baixado
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

### 3. Carregamento dos Dados

Carregamos os arquivos `train.csv` e `test.csv` que contêm os tweets para classificação.

In [3]:
# Carregar os dados
print("Carregando os dados...")
train_df = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
test_df = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv')

# Garantir que não há valores nulos na coluna 'text'
train_df["text"] = train_df["text"].fillna("")
test_df["text"] = test_df["text"].fillna("")

# Mostrar informações básicas
print(f"Tamanho do conjunto de treino: {train_df.shape[0]} exemplos")
print(f"Tamanho do conjunto de teste: {test_df.shape[0]} exemplos")
train_df.head()

Carregando os dados...
Tamanho do conjunto de treino: 7613 exemplos
Tamanho do conjunto de teste: 3263 exemplos


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


### 4. Pré-processamento do Texto

Definimos e aplicamos uma função para limpar os textos dos tweets, removendo URLs, tags HTML, números, pontuação e stopwords (palavras comuns que geralmente não agregam valor semântico).

In [4]:
def preprocess_text(text):
    """Realiza a limpeza e pré-processamento de um texto."""
    if isinstance(text, str):
        # Converter para minúsculas
        text = text.lower()
        
        # Remover URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        
        # Remover HTML tags
        text = re.sub(r'<.*?>', '', text)
        
        # Remover números
        text = re.sub(r'\d+', '', text)
        
        # Remover pontuação
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Remover stopwords
        text = ' '.join(word for word in text.split() if word not in stop_words)
        
        # Remover espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        
        return text
    else:
        return ""

# Aplicar pré-processamento
print("Pré-processando os textos...")
train_df['processed_text'] = train_df['text'].apply(preprocess_text)
test_df['processed_text'] = test_df['text'].apply(preprocess_text)

# Mostrar exemplos de texto processado
print("Exemplo original:", train_df['text'].iloc[0])
print("Exemplo processado:", train_df['processed_text'].iloc[0])

train_df[['text', 'processed_text']].head()

Pré-processando os textos...
Exemplo original: Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Exemplo processado: deeds reason earthquake may allah forgive us


,text,processed_text
0,Our Deeds are the Reason of this #earthquake M...,deeds reason earthquake may allah forgive us
1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,residents asked shelter place notified officer...
3,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders cal...
4,Just got sent this photo from Ruby #Alaska as ...,got sent photo ruby alaska smoke wildfires pou...


### 5. Divisão dos Dados e Vetorização TF-IDF

Dividimos os dados de treino em conjuntos de treino e validação (80% para treino, 20% para validação). Em seguida, usamos `TfidfVectorizer` para converter os textos processados em vetores numéricos.

O TF-IDF (Term Frequency-Inverse Document Frequency) é uma técnica que atribui pesos às palavras baseando-se em sua frequência no documento e na raridade entre todos os documentos. Palavras comuns em um documento mas raras no corpus recebem pesos maiores.

In [5]:
# Separar features (texto processado) e target
X = train_df['processed_text']
y = train_df['target']

# Dividir em treino e validação (80% treino, 20% validação)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {len(X_train)}")
print(f"Tamanho do conjunto de validação: {len(X_val)}")

Tamanho do conjunto de treino: 6090
Tamanho do conjunto de validação: 1523


In [6]:
# Vetorização TF-IDF
print("Aplicando vetorização TF-IDF...")
vectorizer = TfidfVectorizer(
    max_features=5000,  # Limitar o número de features para evitar overfitting
    ngram_range=(1, 1)   # Apenas unigramas nesta abordagem inicial
)

# Ajustar o vetorizador aos dados de treino e transformar treino e validação
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)

print(f"Dimensões da matriz TF-IDF de treino: {X_train_tfidf.shape}")
print(f"Dimensões da matriz TF-IDF de validação: {X_val_tfidf.shape}")

Aplicando vetorização TF-IDF...
Dimensões da matriz TF-IDF de treino: (6090, 5000)
Dimensões da matriz TF-IDF de validação: (1523, 5000)


### 6. Treinamento do Modelo de Regressão Logística

Treinamos um modelo de Regressão Logística usando os vetores TF-IDF. A Regressão Logística é um algoritmo simples e eficiente para classificação binária, que funciona bem com dados de texto vetorizados.

In [7]:
# Treinar o modelo de Regressão Logística
print("Treinando o modelo de Regressão Logística...")
model = LogisticRegression(random_state=42, max_iter=1000) # Aumentar max_iter para garantir convergência
model.fit(X_train_tfidf, y_train)

Treinando o modelo de Regressão Logística...


LogisticRegression(max_iter=1000, random_state=42)

### 7. Avaliação do Modelo

Avaliamos o desempenho do modelo treinado no conjunto de validação, calculando métricas como acurácia, F1-score e gerando um relatório de classificação completo.

In [8]:
# Fazer previsões no conjunto de validação
print("Avaliando o modelo no conjunto de validação...")
y_pred_val = model.predict(X_val_tfidf)

# Calcular métricas
accuracy = accuracy_score(y_val, y_pred_val)
f1 = f1_score(y_val, y_pred_val) # F1-score para a classe positiva (1)
report = classification_report(y_val, y_pred_val)

print(f"Acurácia na validação: {accuracy:.4f}")
print(f"F1-Score (classe 1) na validação: {f1:.4f}")
print("\nRelatório de Classificação:")
print(report)

Avaliando o modelo no conjunto de validação...
Acurácia na validação: 0.8247
F1-Score (classe 1) na validação: 0.7792

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.81      0.90      0.85       869
           1       0.85      0.72      0.78       654

    accuracy                           0.82      1523
   macro avg       0.83      0.81      0.82      1523
weighted avg       0.83      0.82      0.82      1523



### 8. Salvamento do Modelo e Vetorizador

Salvamos o modelo treinado e o vetorizador TF-IDF para uso posterior na geração de previsões para o conjunto de teste ou para aplicações futuras.

In [9]:
# Criar diretório para salvar o modelo se não existir
model_dir = 'logistic_regression_model'
os.makedirs(model_dir, exist_ok=True)

# Salvar modelo e vetorizador
model_path = os.path.join(model_dir, 'logistic_regression_tfidf.pkl')
with open(model_path, 'wb') as f:
    pickle.dump((model, vectorizer), f)

print(f"Modelo e vetorizador salvos em: {model_path}")

Modelo e vetorizador salvos em: logistic_regression_model/logistic_regression_tfidf.pkl


### 9. Geração de Previsões e Arquivo de Submissão

Processamos o conjunto de teste e geramos o arquivo de submissão no formato exigido pelo Kaggle.

In [10]:
# Usar o modelo e vetorizador já em memória
loaded_model = model
loaded_vectorizer = vectorizer

# Preparar os dados de teste
print("Preparando os dados de teste...")
X_test = test_df['processed_text']
X_test_tfidf = loaded_vectorizer.transform(X_test)

print(f"Dimensões da matriz TF-IDF de teste: {X_test_tfidf.shape}")

# Fazer previsões no conjunto de teste
print("Fazendo previsões no conjunto de teste...")
test_predictions = loaded_model.predict(X_test_tfidf)

Preparando os dados de teste...
Dimensões da matriz TF-IDF de teste: (3263, 5000)
Fazendo previsões no conjunto de teste...


In [11]:
# Criar DataFrame de submissão
print("Criando arquivo de submissão...")
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'target': test_predictions
})

# Garantir que os IDs estão na mesma ordem do arquivo de exemplo
submission_df = submission_df.sort_values('id')

# Salvar o arquivo de submissão
submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"Arquivo de submissão salvo em: {submission_path}")
print("Primeiras 10 linhas do arquivo de submissão:")
submission_df.head(10)

Criando arquivo de submissão...
Arquivo de submissão salvo em: submission.csv
Primeiras 10 linhas do arquivo de submissão:


,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


### 10. Conclusão

Este notebook demonstrou a implementação de um modelo de Regressão Logística com vetorização TF-IDF para a classificação de tweets sobre desastres. Esta abordagem simples e eficiente obteve o melhor score (0.79313) entre todas as tentativas, superando inclusive modelos mais complexos como o DistilBERT (0.75543).

Isso demonstra que, em muitos casos, modelos mais simples podem ser mais eficazes do que modelos complexos, especialmente quando o conjunto de dados é relativamente pequeno ou quando as características textuais são bem capturadas por técnicas como TF-IDF.

O arquivo `submission_logistic_regression.csv` está pronto para ser enviado ao Kaggle.